<a href="https://colab.research.google.com/github/financieras/campus/blob/main/pilas/pilas13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Indicés de desorden aplicados
Intentamos mejorar la predicción de qué ```s``` utilizar en función del ```n``` de nuestra pila A, añadiendo unos índices de desorden.  

Vamos a generar diferentes pilas A aleatoriamiente con un cierto ```n``` y ```s```, calculagemos sus índices de desorden y realizaremos un estudio gráfico o de correlación para ver que valor de ```s``` es el óptimo para un cierto ```n``` y unos índices calculados sobre la pila A.

In [4]:
######   ÍNDICES DE DESORDEN   ######
def indice_alejamiento(li):
    total = 0
    for i, v in enumerate(li):
        total += abs(v-i-1)  # se resta 1 porque el índex de un array comienza en 0
    return total

def distancia_contiguos(li):
    total = 0
    n = len(li)
    #print(n)
    for i in range(n):
        if i==0: total += abs(li[i]-li[-1])+abs(li[i]-li[i+1])-n
        elif i==n-1: total += abs(li[i]-li[i-1])+abs(li[i]-li[0])-n
        else: total += abs(li[i]-li[i-1])+abs(li[i]-li[i+1])-2
    return total

def desviaciones(li): # suma de las desviaciones estandar poblaciones entre un valor y su index+1
    total = 0
    for i, v in enumerate(li):
        media = (v+i+1)/2
        total += (((i+1-media)**2 + (v-media)**2)/2)**.5
    return int(total)

In [5]:
######   FUNCIONES   ######
from random import sample, seed
seed()

def lista_ordenada(lis, orden_inverso=False): # la función da True si ponemos lista_ordenada([4,3,2,1], orden_inverso=True)
    ordenada = False
    aux = lis[:]
    aux.sort(reverse=orden_inverso)
    if aux == lis:
        ordenada = True
    return ordenada

def sa(a,b):
    if len(a) > 1: a[0],a[1] = a[1],a[0]
    return a,b
def sb(a,b):
    if len(b) > 1: b[0],b[1] = b[1],b[0]
    return a,b
def ss(a,b):
    sa(a,b)
    sb(a,b)
    return a,b
def pa(a,b):
    if len(b) > 0:
        a.insert(0, b[0])
        b.pop(0)
    return a,b
def pb(a,b):
    if len(a) > 0:
        b.insert(0, a[0])
        a.pop(0)
    return a,b
def ra(a,b):
    if len(a) > 1: a.append(a.pop(0))
    return a,b
def rb(a,b):
    if len(b) > 1: b.append(b.pop(0))
    return a,b
def rr(a,b):
    ra(a,b)
    rb(a,b)
    return a,b
def rra(a,b):
    if len(a) > 1: a.insert(0, a.pop())
    return a,b
def rrb(a,b):
    if len(b) > 1: b.insert(0, b.pop())
    return a,b
def rrr(a,b):
    rra(a,b)
    rrb(a,b)
    return a,b

def separa(tramo, n, s, a, b):   # se llevan de la pila A a la B los números del tramo que toca
    contador = 0
    ancho = int(n/s)
    fin = tramo * ancho          # fin del tramo
    inicio = (tramo-1)*ancho+1   # inicio del tramo
    pasados = 0   # cuenta cuantos números se han pasado de la pila A a la B
    while pasados < fin-inicio+1: # recorremos los elementos de la plia A hasta que pasemos todos los números de este tramo
        if inicio <= a[0] <= fin:
            pb(a,b)
            contador += 1
            pasados += 1
        else:
            ra(a,b)
            contador += 1
    return a, b, contador

def ordena(tramo, n, s, a, b):   # Pasamos de B a A los del tramo que toca, comenzando por el nº mayor, que para el tramo 1 es 5.
    # ordena los números recientemente separados y llevados a B
    # el objetivo es que esos números queden en B ordenados de forma decreciente: 5,4,3,2,1
    contador = 0
    ancho = int(n/s)
    fin = tramo * ancho          # fin del tramo
    inicio = (tramo-1)*ancho+1   # inicio del tramo
    while not lista_ordenada(b, orden_inverso=True): # se repite hasta que B quede ordenada decrecientemente. Si solo queda un elemento en B es que está ordenada
        while max(b) != b[0]:   # buscamos el máximo y vamos haciendo ra o rra hasta que quede el primero
            if b.index(max(b)) <= int(len(b)/2):
                rb(a,b)
            else:
                rrb(a,b)
            contador += 1
        if not lista_ordenada(b, orden_inverso=True):
            pa(a,b); contador += 1
    while len(a) > n-fin:   # llevamos de A a B los números del tramo que toca
        pb(a,b); contador += 1
    return a, b, contador

def ordena_final(tramo, n, s, a, b):    # ordena los números del úlitmo tramo que están en "A" y el objetivo es que queden crecientes: ...,18,19,20.
    contador = 0
    ancho = int(n/s)
    fin = n           # en el último tramo se ajusta para que el último nº sea igual a n
    inicio = (tramo-1)*ancho+1   # inicio del tramo
    while not lista_ordenada(a): # se repite hasta que A quede ordenada en orden creciente. Si solo queda un elemento en A es que está ordenada
        while min(a) != a[0]:
            if a.index(min(a)) <= int(len(a)/2):
                ra(a,b)
            else:
                rra(a,b)
            contador += 1
        if not lista_ordenada(a):
            pb(a,b); contador += 1
    while len(b) > 0:   # llevamos de B a A todos los números que hay en B
        pa(a,b); contador += 1
    return a, b, contador

In [6]:
if __name__ == "__main__":
    for n in [100]:
        for generada in range(1000): # nº de pilas A generadas por cada n
            # Generación de la pila a
            #n = 500   # número de elementos de la pila
            a = sample(range(1, n+1), n)
            a_original = a[:]
            b = []
            ######   PROCEDIMIENTO PRIMERO   ######
            contador = 0
            while not lista_ordenada(a):   # se repite hasta que A quede ordenada. Si solo queda un elemento en A es que está ordenada
                while min(a) != a[0]:   # buscamos el mínimo y vamos haciendo ra o rra hasta que quede el primero
                    if a.index(min(a)) <= int(len(a)/2):
                        ra(a,b)
                    else:
                        rra(a,b)
                    contador += 1
                if not lista_ordenada(a):
                    pb(a,b); contador += 1 #print(a,b) # luego hacemos pb
            while len(b) > 0:   # luego se hace pa pa pa pa hasta que no quede nadie en la pila b
                pa(a,b); contador += 1
            #print("PROCEDIMIENTO PRIMERO contador:", contador)
            primero = contador
            for s in range(2,11):
                ######   PROCEDIMIENTO SEGUNDO   ######
                a = a_original[:]
                n = len(a)
                b = []
                #s = 20   # número de grupos
                contador = 0
                for tramo in range(1, s):   # tramo=1,2,...,s-1. El último tramo se trata de forma diferente
                    a, b, contador_separa = separa(tramo,n,s,a,b)   # lleva de A a B los números correspondientes a ese tramo
                    contador += contador_separa
                    a, b, contador_ordena = ordena(tramo,n,s,a,b)   # ordena los números recientemente separados y llevados a B
                    contador += contador_ordena
                # tramo final donde los números de A ya se ordenan en A, ya no se usa la función separa
                # la pila de trabajo es "A" y el objetivo es que sea creciente: ...,18,19,20.
                a, b, contador_ordena_final = ordena_final(tramo,n,s,a,b)
                contador += contador_ordena_final
                #print("Al final:", a, b)
                #print("PROCEDIMIENTO SEGUNDO contador: ", contador)
                segundo=contador
                #print(f"n: {n}, primero: {primero}, s: {s}, segundo: {segundo}")
                print(n, primero, s, segundo, indice_alejamiento(a_original), distancia_contiguos(a_original), desviaciones(a_original))

Se han truncado las últimas 5000 líneas del flujo de salida.
100 1500 6 868 3226 6040 1613
100 1500 7 897 3226 6040 1613
100 1500 8 900 3226 6040 1613
100 1500 9 901 3226 6040 1613
100 1500 10 948 3226 6040 1613
100 1406 2 900 3212 5916 1606
100 1406 3 891 3212 5916 1606
100 1406 4 859 3212 5916 1606
100 1406 5 891 3212 5916 1606
100 1406 6 877 3212 5916 1606
100 1406 7 832 3212 5916 1606
100 1406 8 832 3212 5916 1606
100 1406 9 876 3212 5916 1606
100 1406 10 910 3212 5916 1606
100 1303 2 870 3304 6564 1652
100 1303 3 799 3304 6564 1652
100 1303 4 774 3304 6564 1652
100 1303 5 824 3304 6564 1652
100 1303 6 819 3304 6564 1652
100 1303 7 881 3304 6564 1652
100 1303 8 885 3304 6564 1652
100 1303 9 875 3304 6564 1652
100 1303 10 912 3304 6564 1652
100 1336 2 883 3628 6960 1814
100 1336 3 843 3628 6960 1814
100 1336 4 829 3628 6960 1814
100 1336 5 894 3628 6960 1814
100 1336 6 907 3628 6960 1814
100 1336 7 909 3628 6960 1814
100 1336 8 933 3628 6960 1814
100 1336 9 919 3628 6960 1814
100 13